In [12]:
# 셀 1: 필요한 패키지 설치
!pip install selenium beautifulsoup4 pandas
!apt-get update
!apt install -y chromium-chromium-driver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

'apt-get'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'apt'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'cp'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [13]:

# 셀 2: 필요한 라이브러리 임포트
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

In [14]:


# 셀 3: Chrome Driver 설정 및 유틸리티 함수
def setup_chrome_driver():
    """
    Colab 환경에 맞는 ChromeDriver 설정
    """
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    return webdriver.Chrome(options=chrome_options)


In [15]:

def scroll_page(driver):
    """
    페이지를 끝까지 스크롤
    """
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


In [16]:

def safe_get(driver, url, max_retries=3):
    """
    안전하게 페이지 로드 시도
    """
    for attempt in range(max_retries):
        try:
            driver.get(url)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
            return True
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"페이지 로드 실패: {url}")
                print(f"오류: {str(e)}")
                return False
            time.sleep(2)


In [17]:

# 셀 4: 메인 스크래핑 함수
def scrape_samsung_faq(urls):
    """
    Samsung FAQ 페이지 스크래핑
    """
    driver = setup_chrome_driver()
    results = []

    try:
        for url in urls:
            if not safe_get(driver, url):
                continue

            scroll_page(driver)
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # FAQ 항목 찾기 (실제 선택자로 수정 필요)
            faq_items = soup.find_all('div', class_='faq-item')

            for item in faq_items:
                try:
                    question = item.find('div', class_='question').text.strip()
                    answer = item.find('div', class_='answer').text.strip()
                    category = item.find('div', class_='category').text.strip()

                    results.append({
                        'category': category,
                        'question': question,
                        'answer': answer,
                        'source_url': url
                    })
                except AttributeError:
                    continue

            time.sleep(2)  # 요청 간 대기

    except Exception as e:
        print(f"스크래핑 중 오류 발생: {str(e)}")

    finally:
        driver.quit()

    return pd.DataFrame(results)


In [18]:

# 셀 5: 실행 코드
if __name__ == "__main__":
    # 스크래핑할 URL 목록
    target_urls = [
        "https://www.samsungsvc.co.kr/solution?category=10471&product=10502",
        "https://www.samsungsvc.co.kr/solution?category=10471&product=10503",
        "https://www.samsungsvc.co.kr/solution?category=10471&product=10504"
           ]

    # 스크래핑 실행
    df_results = scrape_samsung_faq(target_urls)

    # 결과 확인
    print("\n수집된 데이터 미리보기:")
    print(df_results.head())

    # 결과 저장
    df_results.to_csv('samsung_faq.csv', index=False, encoding='utf-8-sig')
    print("\n데이터가 'samsung_faq.csv' 파일로 저장되었습니다.")


수집된 데이터 미리보기:
Empty DataFrame
Columns: []
Index: []

데이터가 'samsung_faq.csv' 파일로 저장되었습니다.
